<a href="https://colab.research.google.com/github/apanand/UChicago-MSADS/blob/main/ML_Final_Project_Spotify_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from numpy import argmax
import pandas as pd
from collections import Counter
from datetime import datetime
import os
import itertools
import math
import collections

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# !pip install ipyplot
# !pip install jupyter-dash
import seaborn as sns
import matplotlib.pyplot as plt

# ## Neural Network
# #!pip install tensorflow
# import tensorflow as tf
# from tensorflow.keras.activations import relu, sigmoid, softmax, tanh, selu, elu
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
# from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# !pip install tensorflow_addons
# from tensorflow.keras.constraints import MaxNorm
# from tensorflow.keras.regularizers import l2, l1
# from tensorboard.plugins.hparams import api as hp
# !pip install keras-tuner
# import kerastuner as kt
# !pip install tensorflow_addons
# import tensorflow_addons as tfa
# !pip install scikeras[tensorflow]
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# import keras_tuner

# Spotify
! pip3 install spotipy
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth


# SkLearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 3.9 MB/s eta 0:00:00


In [4]:
#@title Spotify Developer Credentials { run: "auto", vertical-output: true, display-mode: "form" }
SPOTIFY_CLIENT_ID = "1344d87ca83841e0ab1be63980b9217b" #@param {type:"string"}
SPOTIFY_CLIENT_SECRET = "6ba15a1bd80f4e9c80a8fa90942b453d" #@param {type:"string"}
SPOTIFY_REDIRECT_URI = 'https://www.google.com/'
SCOPE = "playlist-modify-public playlist-read-collaborative playlist-modify-private"
scope = "user-read-recently-played"
assert len(SPOTIFY_CLIENT_ID)>0, 'Please provide a spotify client id'
assert len(SPOTIFY_CLIENT_SECRET)>0, 'Please provide a spotify client secret code'

sp = spotipy.Spotify(
    auth_manager=spotipy.SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope=SCOPE, open_browser=False),
        requests_timeout=20, retries=10)
form_conn = sp.artist('spotify:artist:3jOstUTkEu2JkjvRdBA5Gu')

print('Authorization Sucessful!')

Go to the following URL: https://accounts.spotify.com/authorize?client_id=1344d87ca83841e0ab1be63980b9217b&response_type=code&redirect_uri=https%3A%2F%2Fwww.google.com%2F&scope=playlist-modify-public+playlist-read-collaborative+playlist-modify-private
Enter the URL you were redirected to: https://www.google.com/?code=AQBGvpmjkNzQ0Qvu271xNBrSVhIYJx3M76n5XGAqjiylp8CyBw4SvpKBFL9S62Z4xaHdQlcmjg9gePZx8pdm9X-Sll-CEJgF-tG4VYLlmp-zsuWb-M_MKmlu9EciV9MjBmwACBdwmHo2kp38nZECic7EdgdVMkcsr10ICeUeYRtyS6d-NPP2vW7glm2U6KKJaKUqlH-fyk2oyjtHavOPjaSrok_0PpyEflDeuF8cYVTmk91T7J7-6xuK9tdo8rBTTwcueSXjhGCt
Authorization Sucessful!


In [6]:
def get_all_playlists(sp):
    playlists = []
    response = sp.current_user_playlists()
    while response:
        playlists.extend(response['items'])
        if response['next']:
            response = sp.next(response)
        else:
            response = None
    return playlists

def get_tracks_from_playlist(sp, playlist_id):
    tracks = []
    response = sp.playlist_items(playlist_id)
    while response:
        tracks.extend(response['items'])
        if response['next']:
            response = sp.next(response)
        else:
            response = None
    return tracks

def get_unique_tracks(sp):
    unique_tracks = set()
    playlists = get_all_playlists(sp)
    for playlist in playlists:
        tracks = get_tracks_from_playlist(sp, playlist['id'])
        for item in tracks:
            track = item['track']
            if track:  # Ensure there's a track object
                track_id = track['id']
                if track_id not in unique_tracks:  # Check if track ID is already in the set
                    unique_tracks.add(track_id)
                    print(f"Added {track['name']} by {', '.join(artist['name'] for artist in track['artists'])}")
    return unique_tracks

# Get all unique tracks from your playlists
unique_tracks = get_unique_tracks(sp)
print(f"Total unique tracks: {len(unique_tracks)}")

Added Vanish by Giveon
Added Snowchild by The Weeknd
Added Escape From LA by The Weeknd
Added Summertime by Astral Weeks
Added Teen Scene feat. Buddy by Maeta, Buddy
Added Normal Girl by SZA
Added Pretty Little Birds (feat. Isaiah Rashad) by SZA, Isaiah Rashad
Added If Only by Raveena
Added Look Easy by KAYTRANADA, Lucky Daye
Added Fire for You by Cannons
Added Like I Want You by Giveon
Added Mr. Rager by Kid Cudi
Added Can I Call You Tonight? by Dayglow
Added Look What U Started by The Internet
Added LUCY by Brandon Banks
Added Got It On Me by Pop Smoke
Added Losing My Way by FKJ, Tom Misch
Added Crazy Dream by Tom Misch, Loyle Carner
Added Happiness Over Everything (H.O.E.) (feat. Future & Miguel) by Jhené Aiko, Future, Miguel
Added Just Another Interlude by Bryson Tiller
Added North Face by ODIE
Added Little Lies by ODIE
Added Borderline by Tame Impala
Added Come Around Me by Justin Bieber
Added Maniac by GoldLink
Added Crazy - DRKTMS Remix by Cabu, Love Mansuy, DRKTMS
Added Vex Oh 

In [7]:
def get_track_features(track_ids):
    features_list = []
    index = 0

    # Process in batches of 100 tracks due to API limitations
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        features = sp.audio_features(batch)

        for track, feature in zip(batch, features):
            if feature:  # Ensure the feature is not None
                track_data = sp.track(track)
                feature['title'] = track_data['name']
                feature['all_artists'] = ', '.join([artist['name'] for artist in track_data['artists']])
                feature['popularity'] = track_data['popularity']
                feature['release_date'] = track_data['album']['release_date']
                features_list.append(feature)
            print(f"Processed {index + 1}/{len(track_ids)} tracks.")  # Counter to track progress
            index += 1

    features_df = pd.DataFrame(features_list)
    return features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                        'danceability', 'energy', 'key', 'loudness', 'mode',
                        'acousticness', 'instrumentalness', 'liveness',
                        'valence', 'tempo', 'duration_ms', 'time_signature']]

# Example usage with unique_tracks obtained previously
unique_tracks_list = list(unique_tracks)  # Convert set to list if it's not already
features_df = get_track_features(unique_tracks_list)
print(features_df.head())

Processed 1/4631 tracks.
Processed 2/4631 tracks.
Processed 3/4631 tracks.
Processed 4/4631 tracks.
Processed 5/4631 tracks.
Processed 6/4631 tracks.
Processed 7/4631 tracks.
Processed 8/4631 tracks.
Processed 9/4631 tracks.
Processed 10/4631 tracks.
Processed 11/4631 tracks.
Processed 12/4631 tracks.
Processed 13/4631 tracks.
Processed 14/4631 tracks.
Processed 15/4631 tracks.
Processed 16/4631 tracks.
Processed 17/4631 tracks.
Processed 18/4631 tracks.
Processed 19/4631 tracks.
Processed 20/4631 tracks.
Processed 21/4631 tracks.
Processed 22/4631 tracks.
Processed 23/4631 tracks.
Processed 24/4631 tracks.
Processed 25/4631 tracks.
Processed 26/4631 tracks.
Processed 27/4631 tracks.
Processed 28/4631 tracks.
Processed 29/4631 tracks.
Processed 30/4631 tracks.
Processed 31/4631 tracks.
Processed 32/4631 tracks.
Processed 33/4631 tracks.
Processed 34/4631 tracks.
Processed 35/4631 tracks.
Processed 36/4631 tracks.
Processed 37/4631 tracks.
Processed 38/4631 tracks.
Processed 39/4631 tra

In [35]:
features_df.info() #This is the features df that I have created

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4630 entries, 0 to 4629
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                4630 non-null   object 
 1   title             4630 non-null   object 
 2   all_artists       4630 non-null   object 
 3   popularity        4630 non-null   int64  
 4   release_date      4630 non-null   object 
 5   danceability      4630 non-null   float64
 6   energy            4630 non-null   float64
 7   key               4630 non-null   float64
 8   loudness          4630 non-null   float64
 9   mode              4630 non-null   float64
 10  acousticness      4630 non-null   float64
 11  instrumentalness  4630 non-null   float64
 12  liveness          4630 non-null   float64
 13  valence           4630 non-null   float64
 14  tempo             4630 non-null   float64
 15  duration_ms       4630 non-null   float64
 16  time_signature    4630 non-null   float64


In [15]:
def get_playlist_tracks(sp, playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    while results:
        tracks.extend([item['track'] for item in results['items'] if item['track']])
        results = sp.next(results) if results['next'] else None
    return tracks


def get_track_details_and_features(sp, tracks):
    track_ids = [track['id'] for track in tracks if track['id']]
    features_list = sp.audio_features(track_ids)

    details_list = []
    for track, features in zip(tracks, features_list):
        if features:  # Ensure there are features
            details = {
                'id': track['id'],
                'title': track['name'],
                'all_artists': ', '.join([artist['name'] for artist in track['artists']]),
                'popularity': track['popularity'],
                'release_date': track['album']['release_date'],
                'danceability': features['danceability'],
                'energy': features['energy'],
                'key': features['key'],
                'loudness': features['loudness'],
                'mode': features['mode'],
                'acousticness': features['acousticness'],
                'instrumentalness': features['instrumentalness'],
                'liveness': features['liveness'],
                'valence': features['valence'],
                'tempo': features['tempo'],
                'duration_ms': features['duration_ms'],
                'time_signature': features['time_signature']
            }
            details_list.append(details)
    return details_list

billboard_playlist_id = '6UeSakyzhiEt4NB3UAd6NQ'  # Replace with the actual ID
billboard_tracks = get_playlist_tracks(sp, billboard_playlist_id)
track_details = get_track_details_and_features(sp, billboard_tracks)

# Convert the list of details to a DataFrame
billboard_features_df = pd.DataFrame(track_details)
billboard_features_df.head()

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,97,2024-05-04,0.898,0.472,1,-7.001,1,0.0107,0.000000,0.1410,0.214,101.061,274192,4
1,7fzHQizxTqy8wTXwlrgPQQ,MILLION DOLLAR BABY,Tommy Richman,97,2024-04-26,0.849,0.699,1,-5.106,0,0.0982,0.000215,0.0680,0.927,138.003,155152,4
2,77DRzu7ERs0TX3roZcre7Q,euphoria,Kendrick Lamar,94,2024-04-30,0.831,0.643,1,-5.002,1,0.0460,0.000000,0.0840,0.142,139.948,383639,4
3,6dODwocEuGzHAavXqTbwHv,Fortnight (feat. Post Malone),"Taylor Swift, Post Malone",92,2024-04-19,0.504,0.386,11,-10.976,1,0.5020,0.000015,0.0961,0.281,192.004,228965,4
4,2FQrifJ1N335Ljm3TjTVVf,A Bar Song (Tipsy),Shaboozey,96,2024-04-12,0.722,0.709,9,-4.950,1,0.0633,0.000000,0.0804,0.604,81.012,171292,4


In [18]:
features_df['rating'] = 1

In [19]:
billboard_features_df['rating'] = 0

In [46]:
features_df.to_csv('features_df.csv', index=False)
billboard_features_df.to_csv('billboard_features_df.csv', index=False)

from google.colab import files
files.download("features_df.csv")
files.download('billboard_features_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load DataFrames (assuming 'features_df' and 'billboard_features_df' are already loaded)
features = ['danceability', 'energy', 'key', 'loudness', 'mode',
            'acousticness', 'instrumentalness', 'liveness', 'valence',
            'tempo', 'duration_ms', 'time_signature']

# Normalize feature data
scaler = MinMaxScaler()
features_df[features] = scaler.fit_transform(features_df[features])
billboard_features_df[features] = scaler.transform(billboard_features_df[features])

# Prepare the target by binarizing the popularity (above median popularity)
median_popularity = features_df['popularity'].median()
features_df['target'] = (features_df['popularity'] > median_popularity).astype(int)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(features_df[features], features_df['target'], test_size=0.2, random_state=42)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_dim=len(features)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
93/93 [==============================] - 4s 29ms/step - loss: 0.6961 - accuracy: 0.5022 - val_loss: 0.6912 - val_accuracy: 0.5331
Epoch 2/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6907 - accuracy: 0.5258 - val_loss: 0.6877 - val_accuracy: 0.5614
Epoch 3/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.5393 - val_loss: 0.6880 - val_accuracy: 0.5574
Epoch 4/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6881 - accuracy: 0.5501 - val_loss: 0.6893 - val_accuracy: 0.5371
Epoch 5/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6871 - accuracy: 0.5383 - val_loss: 0.6879 - val_accuracy: 0.5466
Epoch 6/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.5484 - val_loss: 0.6871 - val_accuracy: 0.5425
Epoch 7/50
93/93 [==============================] - 0s 3ms/step - loss: 0.6862 - accuracy: 0.5494 - val_loss: 0.6859 - val_accuracy: 0.5398
Epoch 8/50
93/93 [=

In [47]:
billboard_features_df['predicted_like'] = model.predict(billboard_features_df[features])
top_recommendations = billboard_features_df.sort_values(by='predicted_like', ascending=False).head(20)
print(top_recommendations[['title', 'all_artists', 'predicted_like']])

4/4 [==============================] - 0s 3ms/step
                                  title                    all_artists  \
2                              euphoria                 Kendrick Lamar   
90                              ADIVINO         Myke Towers, Bad Bunny   
24                               greedy                     Tate McRae   
74                       Enough (Miami)                        Cardi B   
94                    whatsapp (wassam)                          Gunna   
39  Wanna Be (with Megan Thee Stallion)  GloRilla, Megan Thee Stallion   
17                          Lovin On Me                    Jack Harlow   
16                             Push Ups                          Drake   
67                  Back Then Right Now                  Tyler Hubbard   
0                           Not Like Us                 Kendrick Lamar   
85                     Si No Quieres No   Luis R Conriquez, Neton Vega   
14      I Can Do It With a Broken Heart                   Tay

In [34]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.6734
Test Accuracy: 0.5875
